#  第8章 数据规整：连接、联合与重塑

## 8.1分层索引 

In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])

In [4]:
data

a  1   -0.872860
   2   -0.448422
   3   -1.889059
b  1   -0.391542
   3   -0.901919
c  1   -0.285951
   2    0.569268
d  2    1.710635
   3    1.692028
dtype: float64

In [5]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

#### 通过分层索引对象，也可以称为部分索引，允许你简洁地选择出数据子集 

In [7]:
data['b']

1   -0.391542
3   -0.901919
dtype: float64

In [8]:
data['b':'c']

b  1   -0.391542
   3   -0.901919
c  1   -0.285951
   2    0.569268
dtype: float64

In [9]:
data.loc[['b','d']]

b  1   -0.391542
   3   -0.901919
d  2    1.710635
   3    1.692028
dtype: float64

In [10]:
data.loc[:,2]

a   -0.448422
c    0.569268
d    1.710635
dtype: float64

####  用unstack 将数据在dataframe中重新排序 

In [11]:
data.unstack()

,1,2,3
a,-0.872860,-0.448422,-1.889059
b,-0.391542,NaN,-0.901919
c,-0.285951,0.569268,NaN
d,NaN,1.710635,1.692028


####  unstack的反操作是stack 

In [12]:
data.unstack().stack()

a  1   -0.872860
   2   -0.448422
   3   -1.889059
b  1   -0.391542
   3   -0.901919
c  1   -0.285951
   2    0.569268
d  2    1.710635
   3    1.692028
dtype: float64

#### 在DataFrame 中，每个轴都可以有分层索引 

In [13]:
frame=pd.DataFrame(np.arange(12).reshape((4,3)),index=[['a','a','b','b'],[1,2,1,2]],columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])

In [14]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

#### 分层的层级可以有名称（可以是字符串或python对象）。如果层级有名称，这些名称会在控制台输出中显示：

In [15]:
frame.index.names=['key1','key2']

In [17]:
frame.columns.names=['state','color']

In [18]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [19]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

####  一个MultiIndex对象可以使用其自身的构造函数创建并复用。前面介绍的带有层级的名称的DataFrame的列可以这样创建：

In [20]:
MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],['Green','Red','Green']],names=['state','color'])

NameError: name 'MultiIndex' is not defined

### 8.1.1重排序和层级排序 

####  有时候，你需要重新排列轴上的层级顺序，或者按照特定层级的值对数据进行排序。swaplevel 接收两个层级序号或层级名称，返回一个进行了层级变更的新对象。但是数据是不变的。

In [21]:
frame.swaplevel('key1','key2') #swap 交换两个层级位置

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

#### 另一方面，sort_index只能在单一层级上对数据进行排序。在进行层级变换时，使用sort_index以使得结果按照层级进行字典排序也很常见： 

In [22]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [23]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 8.1.2按层级进行汇总统计

In [24]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [25]:
frame.sum(level='color',axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

#### 上述例子使用了pandas的groupby机制，这个机制的细节会在本书后续章节讲到 

### 8.1.3 使用DataFrame 的列进行索引 

#### 通常我们不会用DataFrame中的一列或多个列作为索引；反而想要将行索引移动到DataFrame 的列中。下面是示例 

In [27]:
frame=pd.DataFrame({'a':range(7),'b':range(7,0,-1),'c':['one','one','one','two','two','two','two'],'d':[0,1,2,0,1,2,3]})

In [28]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


####  DataFrame的set_index 函数会生成一个新的DataFrame,新的DataFrame使用一个或多个列作为索引：

In [30]:
frame2=frame.set_index(['c','d'])

In [31]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

####  默认情况下这些列会从DataFrame中移除，你也可以将它们留在DataFrame中：

In [32]:
frame.set_index(['c','d'],drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

#### 另一方面，reset_index是set_index的反操作，分层索引的索引层级会被移动到列中： 

In [33]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2联合与合并数据集 

#### 包含在pandas对象的数据可以通过多种方式联合在一起：
#### pandas.merge 根据一个或多个键将行进行连接。
#### pandas.contact 使对象在轴向上进行黏合
#### combine_first 实例方法允许将重叠的数据拼接在一起，以使用一个对象中的值填充另一个对象中的缺失值。

### 8.2.1 数据库风格的DataFrame连接 

In [8]:
df1=pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})

In [9]:
df2=pd.DataFrame({'key':['a','b','d'],'data2':range(3)})

In [10]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [11]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


####  这是一个多对一连接的例子；df1的数据有多个行的标签为a和b,而df2在key列中每个值只有一行 。调用merge处理

In [12]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


#### 请注意我并没有指定在哪一列上进行连接。如果连接的键信息没有指定，merge会自动将重叠列名作为连接的键。但是，显式德指定连接键才是好的实现：

In [13]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


#### 如果每个对象的列名是不同的，你可以分别为他们指定列名：

In [18]:
df3=pd.DataFrame({'1key':['b','b','a','c','a','a','b'],'data1':range(7)})

In [21]:
df4=pd.DataFrame({'rkey':['a','b','d'],'data2':range(3)})

In [22]:
df3

,1key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [23]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [24]:
pd.merge(df3,df4,left_on='1key',right_on='rkey')  #1key和rkey 取交集  默认内连接

,1key,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [25]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [26]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [27]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


### 8.2.2 根据索引合并 

In [29]:
left1=pd.DataFrame({'key':['a','b','a','a','b','c'],'value':range(6)})

In [30]:
right1=pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])

In [31]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [32]:
right1

,group_val
a,3.5
b,7.0


In [37]:
pd.merge(left1,right1,left_on='key',right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


####  由于默认的合并方法是连接键相交，你可以使用外连接来进行合并 

In [38]:
pd.merge(left1,right1,left_on='key',right_index=True,how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


#### 在多层索引数据的情况下，会更复杂，在索引上连接时一个隐式的多键合并

In [39]:
lefth=pd.DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],'key2':[2000,2001,2002,2001,2002],'data':np.arange(5.)})

In [41]:
righth=pd.DataFrame(np.arange(12).reshape((6,2)),index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],[2001,2000,2000,2000,2001,2002]],columns=['event1','event2'])

In [42]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [43]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

#### 这种情况下，你必须以列表的方式指明合并所需多个列（请注意使用how='outer'处理重复的索引值）： 

In [44]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [45]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


#### 使用两边的索引进行合并也是可以的： 

In [48]:
left2=pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],index=['a','c','e'],columns=['Ohio','Nevada'])

In [49]:
right2=pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13,14]],index=['b','c','d','e'],columns=['Missouri','Alabama'])

In [50]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [51]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [53]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


#### DataFrame 有一个方便的join实例方法，用于按照索引合并。该方法也可以用于多个索引相同或相似但没有重叠列的DataFrame对象

In [54]:
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


####  DataFrame的join 方法进行连接键上的左连接，完全保留左边DataFrame的行索引。它还支持在调用DataFrame的某一列上连接传递的DataFrame的索引： 

In [55]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [56]:
right1

,group_val
a,3.5
b,7.0


In [57]:
left1.join(right1,on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [ ]:
对于一些简单索引-索引合并，你可以向join方法传入一个DataFrame列表，这个方法可以替代下一节中更为通用的contact

In [58]:
another=pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],index=['a','c','e','f'],columns=['New York','Oregon'])

In [59]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [60]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [65]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [66]:
left2.join([right2,another],how='outer')

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6369: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


###  8.2.3沿轴向连接

#### 另一种数据组合操作可互换地称为拼接，绑定或堆叠。Numpy的concatenate 函数可以实现该功能： 

In [67]:
arr=np.arange(12).reshape((3,4))

In [68]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [69]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [70]:
s1=pd.Series([0,1],index=['a','b'])

In [71]:
s2=pd.Series([2,3,4],index=['c','d','e'])

In [72]:
s3=pd.Series([5,6],index=['f','g'])

In [73]:
pd.concat([s1,s2,s3])  # 将三个索引不存在的重叠的Series合并时，concat方法会将值与索引粘在一起

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

####  默认情况下，concat 方法是沿着axis=0的轴向生效的，生成另一个Series。如果你传递axis=1,返回的结果是一个DataFrame 

In [74]:
pd.concat([s1,s2,s3],axis=1)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


####  在这个案例中另一个轴向上并没有重叠，你可以看到排序后的索引合集（'out'join外连接）。你也可以传入join='inner': 

In [75]:
s4=pd.concat([s1,s3])

In [76]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [77]:
pd.concat([s1,s4],axis=1)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [78]:
pd.concat([s1,s4],axis=1,join='inner')   #由于join='inner'的选项，‘f’和‘g’标签消失了。

,0,1
a,0,0
b,1,1


####  你甚至可以用join_axes来指定用于连接其他轴向的轴 

In [79]:
pd.concat([s1,s4],axis=1,join_axes=[['a','c','b','e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [80]:
result=pd.concat([s1,s2,s3],keys=['one','two','three'])

In [81]:
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [82]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


####  沿着轴向axis=1 连接Series的时候，keys则成为DataFrame的列头：

In [83]:
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


####  将相同的逻辑拓展到DataFrame对象 

In [84]:
df1=pd.DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],columns=['one','two'])

In [86]:
df2=pd.DataFrame(5+np.arange(4).reshape(2,2),index=['a','c'],columns=['three','four'])

In [87]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [88]:
df2

,three,four
a,5,6
c,7,8


In [91]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'])

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

####  如果你传递的是对象字典而不是列表的话，则字典的键会用于keys选项 

In [93]:
pd.concat({'level1':df1,'level2':df2},axis=1)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

#### 我们可以用names参数命名生成的轴层级： 

In [94]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'],names=['upper','lower'])

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

####  最后需要考虑的是行索引不包括任何相关数据的DataFrame: 

In [95]:
df1=pd.DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])

In [96]:
df2=pd.DataFrame(np.random.randn(2,3),columns=['b','d','a'])

In [97]:
df1

,a,b,c,d
0,-1.294054,0.944731,0.992036,0.678006
1,-1.692460,-0.113304,0.266667,-0.748276
2,-0.824394,0.898844,0.615976,1.310475


In [98]:
df2

,b,d,a
0,0.224940,-0.459484,-1.571539
1,-1.509911,-1.292313,-0.131632


####   在这个示例中，我们传入ignore_index=True

In [99]:
pd.concat([df1,df2],ignore_index=True)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-1.294054,0.944731,0.992036,0.678006
1,-1.692460,-0.113304,0.266667,-0.748276
2,-0.824394,0.898844,0.615976,1.310475
3,-1.571539,0.224940,NaN,-0.459484
4,-0.131632,-1.509911,NaN,-1.292313


In [100]:
pd.concat([df1,df2],ignore_index=False)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-1.294054,0.944731,0.992036,0.678006
1,-1.692460,-0.113304,0.266667,-0.748276
2,-0.824394,0.898844,0.615976,1.310475
0,-1.571539,0.224940,NaN,-0.459484
1,-0.131632,-1.509911,NaN,-1.292313


### 8.2.4联合重叠数据

####  还有一个数据联合场景，既不是合并也不是连接。你可能有两个数据集，这两个数据集的索引全部或部分重叠。作为示例，考虑numpy的where 函数，这个函数可以进行面向数组的if-else等价操作： 

In [4]:
a=pd.Series([np.nan,2.5,0.0,3.5,4.5,np.nan],index=['f','e','d','c','b','a'])

In [5]:
b=pd.Series([0.,np.nan,2.,np.nan,np.nan,5],index=['a','b','c','d','e','f'])

In [103]:
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [104]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [6]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [7]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

In [8]:
df1=pd.DataFrame({'a':[1.,np.nan,5.,np.nan],'b':[np.nan,2.,np.nan,6.],'c':range(2,18,4)})

In [9]:
df2=pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],'b':[np.nan,3.,4.,6.,8.]})

In [10]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [11]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [12]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


##  8.3重塑和透视

### 8.3.1使用多层索引进行重塑 

#### stack 堆叠，会旋转或将列中的数据透视到行
#### unstack 拆堆， 会将行中的数据透视到列

In [14]:
data=pd.DataFrame(np.arange(6).reshape((2,3)),index=pd.Index(['Ohio','Colorado'],name='state'),columns=pd.Index(['one','two','three'],name='number'))

In [15]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [16]:
result=data.stack()

In [17]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

####  从一个多层索引序列中，你可以使用unstack方法将数据重排列后放入一个DataFrame中： 

In [18]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


####  默认情况下，最内层是已拆堆的（与stack方法一样）。你可以传入一个层级序号或名称来拆分一个不同的层级 

In [19]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [20]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


####  如果层级中所有值并未包含于每个子分组中，拆分可能会引入缺失值： 

In [21]:
s1=pd.Series([0,1,2,3],index=['a','b','c','d'])

In [22]:
s2=pd.Series([4,5,6],index=['c','d','e'])

In [23]:
data2=pd.concat([s1,s2],keys=['one','two'])

In [24]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [25]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


####  默认情况下，堆叠会过滤出缺失值，因此堆叠拆堆的操作是可逆的： 

In [26]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [27]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [28]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

####  当你在DataFrame 中拆堆时，被拆堆的层级会变成结果中最低的层级：

In [35]:
df=pd.DataFrame({'left':result,'right':result+5},columns=pd.Index(['left','right'],name='side'))

In [31]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [36]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [37]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

#### 在调用stack 方法时，我们可以指明需要堆叠的轴向名称：

In [39]:
 df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### 8.3.2 将"长"透视为宽 

In [40]:

data=pd.read_csv('D:/data_analysis/bookcode/pydata-book/examples/macrodata.csv')

In [41]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [42]:
periods=pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')

In [43]:
columns=pd.Index(['realgdp','influ','unemp'],name='item')

In [44]:
data=data.reindex(columns=columns)

In [45]:
data.index=periods.to_timestamp('D','end')

In [47]:
ldata=data.stack().reset_index().rename(columns={0:'value'})

In [49]:
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,unemp,5.800
2,1959-06-30,realgdp,2778.801
3,1959-06-30,unemp,5.100
4,1959-09-30,realgdp,2775.488
5,1959-09-30,unemp,5.300
6,1959-12-31,realgdp,2785.204
7,1959-12-31,unemp,5.600
8,1960-03-31,realgdp,2847.699
9,1960-03-31,unemp,5.200


In [50]:
pivoted=ldata.pivot('date','item','value')

In [51]:
pivoted

item,realgdp,unemp
date,,
1959-03-31,2710.349,5.8
1959-06-30,2778.801,5.1
1959-09-30,2775.488,5.3
1959-12-31,2785.204,5.6
1960-03-31,2847.699,5.2
1960-06-30,2834.390,5.2
1960-09-30,2839.022,5.6
1960-12-31,2802.616,6.3
1961-03-31,2819.264,6.8


In [52]:
ldata['value2']=np.random.randn(len(ldata))

In [53]:
ldata[:10]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,1.972565
1,1959-03-31,unemp,5.800,0.004866
2,1959-06-30,realgdp,2778.801,-0.352989
3,1959-06-30,unemp,5.100,-0.497151
4,1959-09-30,realgdp,2775.488,-0.449436
5,1959-09-30,unemp,5.300,2.115437
6,1959-12-31,realgdp,2785.204,-1.076463
7,1959-12-31,unemp,5.600,-0.397329
8,1960-03-31,realgdp,2847.699,-0.090517
9,1960-03-31,unemp,5.200,0.339752


#### 如果遗漏最后一个参数，你会得到一个含有多层列的DataFrame:

In [55]:
pivoted=ldata.pivot('date','item')

In [56]:
pivoted[:5]

value          value2          
item         realgdp unemp   realgdp     unemp
date                                          
1959-03-31  2710.349   5.8  1.972565  0.004866
1959-06-30  2778.801   5.1 -0.352989 -0.497151
1959-09-30  2775.488   5.3 -0.449436  2.115437
1959-12-31  2785.204   5.6 -1.076463 -0.397329
1960-03-31  2847.699   5.2 -0.090517  0.339752

In [57]:
pivoted['value'][:5]

item,realgdp,unemp
date,,
1959-03-31,2710.349,5.8
1959-06-30,2778.801,5.1
1959-09-30,2775.488,5.3
1959-12-31,2785.204,5.6
1960-03-31,2847.699,5.2


####  请注意pivot方法等价于使用set_index创建分层索引，然后调用unstack: 

In [59]:
unstacked=ldata.set_index(['date','item']).unstack('item')

In [60]:
unstacked[:7]

value          value2          
item         realgdp unemp   realgdp     unemp
date                                          
1959-03-31  2710.349   5.8  1.972565  0.004866
1959-06-30  2778.801   5.1 -0.352989 -0.497151
1959-09-30  2775.488   5.3 -0.449436  2.115437
1959-12-31  2785.204   5.6 -1.076463 -0.397329
1960-03-31  2847.699   5.2 -0.090517  0.339752
1960-06-30  2834.390   5.2  2.316631 -1.479456
1960-09-30  2839.022   5.6  0.878704  0.852964

###  将“宽”透视为“长”

#### pivot方法的反操作时pandas.melt。与将一列变换为新的dataframe中的多列不同，它将多列合并为一列，产生一个新的DataFrame,其长度比输入更长。

In [63]:
df=pd.DataFrame({'key':['foo','bar','baz'],'A':[1,2,3],'B':[4,5,6],'C':[7,8,9]})

In [64]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [65]:
melted=pd.melt(df,['key'])

In [66]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


#### 使用melt方法，可以将数据重塑回原先的布局： 

In [67]:
reshaped=melted.pivot('key','variable','value')

In [68]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [69]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [70]:
pd.melt(df,id_vars=['key'],value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [71]:
pd.melt(df,value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [72]:
pd.melt(df,value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
